<a href="https://www.kaggle.com/code/kurshidbasheer/eda-stanford-rna3d-folding-2?scriptVersionId=298439061" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os

print("Top level folders:")
print(os.listdir('/kaggle/input'))


In [ ]:
os.listdir('/kaggle/input/stanford-rna-3d-folding-2')


In [ ]:
train_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv')
train_labels.head()

In [ ]:
train_labels['copy'].unique()

In [ ]:
train_labels.groupby('ID').size().head()

In [ ]:
train_labels.groupby('copy').size()

In [ ]:
#imp
train_labels.groupby(['ID','copy']).size().head()

Pandas detected mixed datatype in column 6

In [ ]:
train_labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

No of copies for a single sequence

In [ ]:
train_labels['seq_id'] = train_labels['ID'].str.split('_').str[0]

train_labels.groupby('seq_id')['copy'].nunique().describe()

train_labels.groupby('seq_id')['copy'].max().describe()

Analyze Sequence Length Distribution

In [ ]:
train_seq = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv')

train_seq['length'] = train_seq['sequence'].str.len()

train_seq['length'].describe()


In [ ]:
train_seq['length'].hist(bins=50)


How many sequences are truly massive?

In [ ]:
(train_seq['length'] > 5000).sum()


In [ ]:
(train_seq['length'] > 10000).sum()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
    len(os.listdir('/kaggle/input/stanford-rna-3d-folding-2/PDB_RNA'))


In [ ]:
train = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv')
test = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/test_sequences.csv')

print("Train max length:", train['sequence'].apply(len).max())
print("Test max length:", test['sequence'].apply(len).max())


The error tells us something important:

'sequence_id' is not a column name in train_labels.csv.

In [ ]:
labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

print(labels.columns)


In [ ]:
labels.head()

inspect actual column names in train_labels.csv

How many sequences in train are >5000 nt?

In [ ]:
(train['sequence'].apply(len) > 5000).sum()


Compute Labeled Residues Per Sequence

In [ ]:
labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

labels_per_seq = labels.groupby('ID').size()

print("Max labeled residues:", labels_per_seq.max())
print("Median labeled residues:", labels_per_seq.median())
print("Min labeled residues:", labels_per_seq.min())
print("Number of sequences:", labels_per_seq.shape[0])

how many conformations per RNA?

In [ ]:
labels['copy'].value_counts()

In [ ]:
labels.head(10)


In [ ]:
labels['ID'].head(10)


In [ ]:
labels['seq_id'] = labels['ID'].apply(lambda x: x.split('_')[0])


In [ ]:
labels_per_seq = labels.groupby('seq_id').size()

print("Max labeled residues:", labels_per_seq.max())
print("Median labeled residues:", labels_per_seq.median())
print("Min labeled residues:", labels_per_seq.min())
print("Number of sequences:", labels_per_seq.shape[0])


In [ ]:
(train_seq['length'] < 300).sum()
(train_seq['length'] < 1000).sum()


In [ ]:
import pandas as pd

train_seq = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv'
)

train_seq['length'] = train_seq['sequence'].apply(len)


In [ ]:
print("Total sequences:", len(train_seq))

print("< 300 nt:", (train_seq['length'] < 300).sum())
print("< 1000 nt:", (train_seq['length'] < 1000).sum())
print("< 2000 nt:", (train_seq['length'] < 2000).sum())
print("> 5000 nt:", (train_seq['length'] > 5000).sum())


There are extreme outliers (125k nt)

Those are rare (very few sequences).
Likely ribosomal RNA.

You must confirm how many such extreme cases exist:

check if P or c4 exists (forchirality constraints)

In [ ]:
type(train_seq)
train_seq.columns
train_seq.head(1)

In [ ]:
print(train_labels.columns)


In [ ]:
import os

# IDs from train_sequences.csv
seq_ids = set(train_seq["target_id"].str.upper())

# IDs from rna pdb folder
cif_ids = set([f.split(".")[0].upper()
               for f in os.listdir("/kaggle/input/stanford-rna-3d-folding-2/PDB_RNA")
               if f.endswith(".cif")])

print("Total sequence IDs:", len(seq_ids))
print("Total CIF IDs:", len(cif_ids))

print("Overlap:", len(seq_ids & cif_ids))
print("Missing in CIF:", len(seq_ids - cif_ids))
print("Extra in CIF:", len(cif_ids - seq_ids))


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv")

print(df.shape)
print(df.columns)
df.tail()


/tmp/ipykernel_55/2243670145.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv")


(7794971, 8)
Index(['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'chain', 'copy'], dtype='object')


,ID,resname,resid,x_1,y_1,z_1,chain,copy
7794966,9ZC8_1436,G,1436,244.609,370.637,308.076,A,2
7794967,9ZC8_1437,A,1437,246.970,365.282,307.269,A,2
7794968,9ZC8_1438,A,1438,249.310,360.899,309.337,A,2
7794969,9ZC8_1439,U,1439,250.751,358.343,313.544,A,2
7794970,9ZC8_1440,U,1440,250.868,356.799,318.639,A,2


To decide whether high-copy RNAs are:

True flexible conformations
or

NMR ensembles (multiple models of same structure)

you must inspect metadata inside the .cif files.

This distinction matters a lot.

Inspecting RNA metadata

In [4]:
import pandas as pd

meta = pd.read_csv("/kaggle/input/stanford-rna-3d-folding-2/extra/rna_metadata.csv")
print(meta.columns)
meta.head()


Index(['target_id', 'pdb_id', 'chain_id', 'auth_chain_id', 'sequence',
       'entity_id', 'entity_type', 'unmapped_canonical', 'unmapped_nakb',
       'mapped_fraction', 'undefined_residues', 'unexpected_residues',
       'nonstandard_residues', 'canonical_sequence', 'full_sequence',
       'sequence_expected', 'sequence_observed', 'missing_residues',
       'keyword_ribosome', 'composition_total_mass', 'composition_rna_mass',
       'composition_rna_fraction', 'composition_na_hybrid_mass',
       'composition_na_hybrid_fraction', 'assembly_num_assemblies',
       'assembly_assembly_defined', 'assembly_has_symmetry_operators',
       'assembly_rna_entities', 'assembly_rna_copies',
       'assembly_na_hybrid_entities', 'assembly_na_hybrid_copies',
       'temporal_cutoff', 'resolution', 'method', 'title', 'group_id',
       'seq_group_id', 'mmseqs_0.300', 'mmseqs_0.400', 'mmseqs_0.500',
       'mmseqs_0.600', 'mmseqs_0.700', 'mmseqs_0.800', 'mmseqs_0.900',
       'mmseqs_0.950', 'rna3d

,target_id,pdb_id,chain_id,auth_chain_id,sequence,entity_id,entity_type,unmapped_canonical,unmapped_nakb,mapped_fraction,...,inter_chain_structuredness,intra_chain_structuredness,total_structuredness,length,length_observed,length_expected,fraction_observed,inter_chain_structuredness_adjusted,intra_chain_structuredness_adjusted,total_structuredness_adjusted
0,4TNA_A,4TNA,A,A,GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGG...,1,rna,False,False,0.184,...,0.0,0.934,0.934,76,76,76,1.0,0.0,0.934,0.934
1,6TNA_A,6TNA,A,A,GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGG...,1,rna,False,False,0.184,...,0.0,0.947,0.947,76,76,76,1.0,0.0,0.947,0.947
2,1TRA_A,1TRA,A,A,GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGG...,1,rna,False,False,0.184,...,0.0,0.934,0.934,76,76,76,1.0,0.0,0.934,0.934
3,1TN2_A,1TN2,A,A,GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGG...,1,rna,False,False,0.184,...,0.0,0.947,0.947,76,76,76,1.0,0.0,0.947,0.947
4,1TN1_A,1TN1,A,A,GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGG...,1,rna,False,False,0.184,...,0.0,0.947,0.947,76,76,76,1.0,0.0,0.947,0.947


In [5]:
meta['method'].value_counts()


method
EM                  12626
X-ray               12608
NMR                  1004
Multiple methods       16
Other                   1
Name: count, dtype: int64

In [7]:
meta['assembly_rna_copies'].describe()


count    26255.000000
mean         5.079033
std          8.424271
min          0.000000
25%          2.000000
50%          3.000000
75%          6.000000
max         79.000000
Name: assembly_rna_copies, dtype: float64

In [10]:
meta['assembly_rna_copies'].value_counts().head(10)

assembly_rna_copies
1     5745
2     5178
7     3359
5     2599
6     2493
3     2420
4     1894
8      446
0      442
10     220
Name: count, dtype: int64